<a href="https://colab.research.google.com/github/bjimenezTechnodomus/licitaciones/blob/main/clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

import pandas as pd
import re

from google.colab import data_table
data_table.enable_dataframe_formatter()
gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
worksheet = gc.open('Total partidas').sheet1

rows = worksheet.get_all_values()

data = pd.DataFrame.from_records(rows)
data.columns = data.iloc[0]
data.drop(0, inplace=True)
display(data.info())
display(data.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112180 entries, 1 to 112180
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   Id_Partida                112180 non-null  object
 1   Archivo_Origen            112180 non-null  object
 2   Consecutivo               112180 non-null  object
 3   Número del Procedimiento  112180 non-null  object
 4   Tipo de Procedimiento     112180 non-null  object
 5   Convocante                112180 non-null  object
 6   Par- tida                 112180 non-null  object
 7   Cantidad Mínima           112180 non-null  object
 8   Cantidad Máxima           112180 non-null  object
 9   Unidad                    112180 non-null  object
 10  Descripción               112180 non-null  object
 11  Descripción de la Clave   112180 non-null  object
 12  Observaciones             112180 non-null  object
 13  Grupo de productos        112180 non-null  object
 14  Esta

None

,Id_Partida,Archivo_Origen,Consecutivo,Número del Procedimiento,Tipo de Procedimiento,Convocante,Par- tida,Cantidad Mínima,Cantidad Máxima,Unidad,Descripción,Descripción de la Clave,Observaciones,Grupo de productos,Estaba equivocada,Cuadro Básico,Nos interesa,Razones,Grupo Productos
count,112180,112180,112180,112180,112180,112180,112180,112180,112180,112180,112180,112180,112180,112180,112180,112180,112180,112180,112180
unique,110625,55,5428,3820,5,440,1184,1267,5186,3413,78275,13,4,14,4,14694,2,50,35
top,1oKJURJWilHXXaAwCrmoW1F4KiZ0Flimb9vLP8hW-xCY00...,1OAk_gdJojr40Hi2f8FNzChB_Psr7ZQYQ82Nw7ozUUpI,696,LA-008IZC999-E632-2021,Adjudicación Directa,IMSS-Coordinación de Abastecimiento y Equipami...,1,,1,,"060.168.6637.11 SONDAS DE LATEX, PUNTA REDONDA...",Material de Curación,,No incluye,No,,,,
freq,2,5428,57,1089,43991,5304,3443,95071,19549,17602,57,63076,111511,110012,111570,43028,92865,83746,110466


In [ ]:
data = data[['Id_Partida', 'Convocante', 'Descripción', 'Descripción de la Clave', 'Grupo de productos', 'Cuadro Básico', 'Nos interesa', 'Grupo Productos']]
data.columns = ['id', 'convocante', 'descripcion', 'clave', 'grupo_1', 'cuadro_basico', 'interes', 'grupo_2']
data['cuadro_2'] = data['descripcion'].str.extract(r'(\d{3}[\.\s-]\d{3}[\.\s-]\d{4})').replace(r'[\s-]', '.')
CATEGORIAS = ['grupo_1', 'clave', 'interes', 'grupo_2']
data['grupo_2'] = data['grupo_2'].str.lower()
data['descripcion'] = data['descripcion'].str.lower()
data['grupo_2'][data['grupo_2'] == 'desinfectantes'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'contenedores'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'lavado y cuidado de instrumental'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'lavadoras termodesinfectoras'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'película y químicos dental'] = 'filme plano'
data['grupo_2'][data['grupo_2'] == 'lavadoras ultrasónicas'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'lavadoras ultrasonicas'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'reveladoras y casetas análogas'] = 'filme plano'
data['grupo_2'][data['grupo_2'] == 'bolsa / rollo material no tejido'] = 'envoltura material no tejido'
data['grupo_2'][data['grupo_2'] == 'cr'] = 'rayos x digital'
data['grupo_2'][data['grupo_2'] == 'mesa de rayos x'] = 'rayos x digital'
data['grupo_2'][data['grupo_2'] == 'mastografias'] = 'mastografía'
data['grupo_2'][data['grupo_2'] == 'pelicula dry'] = 'película dry'
data.loc[49804, 'grupo_2'] == 'indicadores biológicos'
data.loc[72386, 'grupo_2'] == 'indicadores biológicos'
data['grupo_2'][data['grupo_2'] == 'retos ind. quimico'] = 'indicadores biológicos'
data['grupo_2'][data['grupo_2'] == 'bolsa / rollo material no tejido'] = 'envoltura material no tejido'
data[CATEGORIAS] = data[CATEGORIAS].apply(lambda x: x.astype('category'), axis=0)
categorias_dumies = pd.get_dummies(data[CATEGORIAS])
grupo_tech = pd.get_dummies(data['grupo_1'])

In [ ]:
data['grupo_2'].value_counts()

                                110466
bolsa / rollo mixta papel          740
indicadores biológicos             198
vapor                              115
indicadores químicos               114
plasma                             108
mastografía                         88
indicadores biologicos              52
lavado                              51
rayos x digital                     40
selladoras                          36
envoltura material no tejido        35
bolsa / rollo tyvek                 32
indicadores quimicos                26
película dry                        24
filme plano                         20
incubadoras                         19
licenciamiento                      16
Name: grupo_2, dtype: int64

In [ ]:
display(data.loc[49804])
display(data.iloc[49804])

id               1jB__6DfrGci8NtX9tR6F33N1qcaZU3g_WBGTIw8Y4w001...
convocante       IMSS-Coordinación de abastecimiento y equipami...
descripcion      060.711.0046.11 testigos controles biologicos ...
clave                                         Material de Curación
grupo_1                                                      Vapor
cuadro_basico                                            607110046
interes                                                           
grupo_2                                     indicadores biológicos
cuadro_2                                              060.711.0046
Name: 49804, dtype: object

id               1jB__6DfrGci8NtX9tR6F33N1qcaZU3g_WBGTIw8Y4w001...
convocante       IMSS-Coordinación de abastecimiento y equipami...
descripcion      080.909.5557.11 tubo sistema para toma y recol...
clave                                      Material de Laboratorio
grupo_1                                                 No incluye
cuadro_basico                                            809095557
interes                                                           
grupo_2                                                           
cuadro_2                                              080.909.5557
Name: 49805, dtype: object

In [ ]:
y = data['grupo_2']
X = data.drop('grupo_2', axis=1)
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X, y):
  X_train = X.iloc[train_index]
  X_test = X.iloc[test_index]
  y_train = y.iloc[train_index]
  y_test = y.iloc[test_index]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# El extractor de tokens segun un patron alfanumerico
tokens = '[A-Za-z0-9]+(?=\\s+)'

X_train['descripcion'].fillna('', inplace=True)

# Instanciacion del vectorizador
vec_alpha = CountVectorizer(token_pattern= tokens)

# Aplicación del vectorizador
vec_alpha.fit_transform(X_train['descripcion'])

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


<89744x32555 sparse matrix of type '<class 'numpy.int64'>'
	with 1479504 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

pl = Pipeline([
               ('vec', CountVectorizer(tokens)),
               ('clf', OneVsRestClassifier(LogisticRegression(max_iter=200)))
])

pl.fit(X_train['descripcion'], y_train)
accuracy = pl.score(X_test['descripcion'], y_test)
display(accuracy)

0.9967908718131574

# Clasificación con Spacy

Una prueba para mejorar el modelo usando los modelos preentrenados de spacy

In [ ]:
# Ejemplo usando Spacy para la clasificacción de las partidas
!pip install --upgrade spacy
!python -m spacy download es_core_news_sm
import spacy
nlp = spacy.load("es_core_news_sm")

In [ ]:
nlp = spacy.blank('es')
textcat = nlp.add_pipe('textcat')
labels = data['grupo_2'].unique()
for label in labels:
  textcat.add_label(label)

train_text = data['descripcion'].values
train_labels = [{'cats': {label: case == label for label in labels}} for case in data['grupo_2']]

In [ ]:
train_data = list(zip(train_text, train_labels))
train_data[:3]

[('(suministros medicos ren.1) prueba de efectiva de analisis clinicos centro gestor 0101 solicitud de pedido 0190038268 00010 agrup. carrito 3000020779 0001 entrega en: carretera estatal leon cueramaro km 7.5 s/n c.p. 37680 leon los ramirez méxico/gto descripción técnica: prueba efectiva de analisis clinicos generales: pruebas solicitadas:quimica sanguinea de iv elementos (glucosa, urea, creatinina y acido urico)perfil de lipidos (colesterol total, trigliceridos, colesterol_ldl y colesterol_hdlinsumos: el proveedor debera entregar todos los reactivos, controles, calibradores e insumos necesarios para que el laboratorio pueda realizar el numero de pruebas requeridas.-se solicita que todos los reactivos e insumos al momento de la e',
  {'cats': {'': True,
    'bolsa / rollo mixta papel': False,
    'bolsa / rollo tyvek': False,
    'envoltura material no tejido': False,
    'filme plano': False,
    'incubadoras': False,
    'indicadores biologicos': False,
    'indicadores biológicos':

In [ ]:
from spacy.util import minibatch
from spacy.training.example import Example 

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

batches = minibatch(train_data, size=10)
for batch in batches:
  for text, labels in batch:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, labels)
    nlp.update([example], sgd=optimizer)

In [ ]:
text = "806101481 (partida 1 sustancias quimicas ren.2) placa de agar gelosa chocolate, desechable esteril de poliestireno cristal de 92 mm de diametro con 18 ml. de medio hidratado con 1 % de hemoglobina y 1 % de suplemento nutritivo. apilable, con identificación impresa individual. bolsa estéril de pvdc polipropileno sellada al calor con etiqueta de identificación. caducidad 14 semanas. placa. rtc placa c/1"
doc = nlp.tokenizer(text)
textcat = nlp.get_pipe("textcat")
scores = textcat.predict(doc)
scores

NameError: ignored

## Ejemplo de aplicación del modelado

Este es un ejemplo del uso del algoritmo en un archivo anterior de partidas.

In [ ]:
ejemplo = gc.open_by_key('1MJEn7aqCjBaMq2EQ2x1VhKgRZkgKeXpKQw0VFQ8P9HQ')
ejemplo = ejemplo.get_worksheet(1)
rows = ejemplo.get_all_values()
ejemplo = pd.DataFrame.from_records(rows)
ejemplo.columns = ejemplo.iloc[0]
ejemplo.drop(0, inplace=True)
ejemplo['Descripción'] = ejemplo['Descripción'].str.lower()
display(ejemplo.describe())

NameError: ignored

In [ ]:
ejemplo['prediccion'] = pl.predict(ejemplo['Descripción'])
ejemplo

,Número del Procedimiento,Tipo de Procedimiento,Convocante,Par- tida,Cantidad Mínima,Cantidad Máxima,Unidad,Descripción,Descripción de la Clave,Observaciones,prediccion
1,LP-919044992-I35-2017,Licitación,SERVICIOS DE SALUD DE NUEVO LEON 0.P.D.,1,,24,EQUIPO C/96,800812141 (partida 1 sustancias quimicas ren.1...,Material de Laboratorio,,
2,LP-919044992-I35-2017,Licitación,SERVICIOS DE SALUD DE NUEVO LEON 0.P.D.,2,,7,PLACA C/1,806101481 (partida 1 sustancias quimicas ren.2...,Material de Laboratorio,,
3,LP-919044992-I35-2017,Licitación,SERVICIOS DE SALUD DE NUEVO LEON 0.P.D.,3,,7,EQUIPO C/96,800702508 (partida 1 sustancias quimicas ren.3...,Material de Laboratorio,,
4,LP-919044992-I35-2017,Licitación,SERVICIOS DE SALUD DE NUEVO LEON 0.P.D.,4,,4,TUBOS C/10,5019994934 (partida 1 sustancias quimicas ren....,Material de Laboratorio,,
5,LP-919044992-I35-2017,Licitación,SERVICIOS DE SALUD DE NUEVO LEON 0.P.D.,5,,2,FRASCO C/1,800810590 (partida 1 sustancias quimicas ren.5...,Material de Laboratorio,,
...,...,...,...,...,...,...,...,...,...,...,...
766,AA-019GYR079-E160-2017,Adjudicación Directa,"IMSS-Unidad Médica de Alta Especialidad, Hospi...",4,,12,25 CJA RLL,060.203.0397.11.01 cintas para uso quirurgico ...,Material de Curación,,
767,AA-019GYR079-E161-2017,Adjudicación Directa,"IMSS-Unidad Médica de Alta Especialidad, Hospi...",1,,20,16 ENV LTO,060.066.0773.12.01 antisepticos. alcohol desna...,Material de Curación,,
768,AA-019GYR079-E161-2017,Adjudicación Directa,"IMSS-Unidad Médica de Alta Especialidad, Hospi...",2,,1,33491 PZA PZA,"060.681.0059.11.01 pavales de forma anatomica,...",Material de Curación,,
769,AA-03890O001-E211-2017,Adjudicación Directa,CIBNOR-Subdirección de Recursos Materiales #03...,1,,1,,(mat. laboratorio ren.1) (sticky stuff) 1 shee...,Material de Laboratorio,,


# Procesado de los datos de Cantidades

In [ ]:
textos = [text for text in data['Cantidad Máxima'] if len(text) > 5]

rango_maxmin = r'(\d+) a (\d+)'
cantidad_unidad = r'(\d+) (\w+)'
minmax = r'min+max'
dos_palabras = r'([A-Z]+) +([A-Z]+)'
miles = r'(\d{1,3}),(\d{3})'

display(textos)

In [ ]:
display([re.match(cantidad_unidad, texto).group(2) for texto in textos if re.match(cantidad_unidad, texto)])

In [ ]:
# Funciones utilitarias

def num_text_to_int(text):
  nums = ['cero', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis', 'siete', 'ocho', 'nueve', 'diez']

In [ ]:
data['Cantidad Máxima'] = (data['Cantidad Máxima'].str.replace('\D', ''))
data['Cantidad Mínima'] = (data['Cantidad Mínima'].str.replace('\D', ''))
data['Cantidad Máxima'] = data['Cantidad Máxima'].astype(int)
data['Cantidad Mínima'] = data['Cantidad Mínima'].astype(int)

In [ ]:
for index, num in enumerate(data['Cantidad Máxima']):
  print('{} {}'.format(index, int(num)))